# 1. Producing the data
## implement one Apache Kafka producer to simulate the real-time data transfer from one repository to another.


## Configuration

- if using pandas to read csv, need to specify dtype=str or need to transform them back to String before sending -> otherwise, pandas would try to infer schema for each column

## Kafka Producer Methods

In [4]:
# import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import csv
import pandas as pd
import datetime
import pprint as pp
from datetime import timedelta
import json
from json import dumps

#configuration
#home ip
hostip = "192.168.8.133"
#uni up
# hostip = "118.138.78.178"

def read_file(file):
    """
    Reads a file into a pandas dataframe
    """
    df = pd.read_csv(file,encoding='utf-8',dtype=str)
    #parse date
    df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True)
    #create mask of year 2011 and sort values
    df = (df[df['Date'].dt.year == 2011]
          .sort_values(['Date'],
                    ascending=[True]))
    return df

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  #dumps passes a dictionary into a string
                                  #encondes the string as ascii
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

if __name__ == '__main__':
    
    #setup
    topic = 'assignment2b'
    file = read_file('data/produce_data.csv')
    producer = connect_kafka_producer()
    start_date = min(file['Date'])
    end_date = max(file['Date'])
    delta = timedelta(weeks=1)
    print('Publishing records..')
    while True:
        #if start_date hasn't reached end_date, print test
        if start_date <= end_date:
            ts =dt.datetime.now().timestamp()
            #slide dataframe
            data = (file.loc[file['Date']==start_date,:]
                    #cast all columns to string
                    .astype(str)
                    #convert to dictionary
                    .to_dict('records'))
            #cast date to str.
        #else, reset the start_date and continue from the top
        else:
            start_date = min(file['Date'])
            continue
        #add one more week to check
        start_date += delta
        # append data
        data = {'ts':ts,'data':data}
        #publish message
        publish_message(producer,topic,dumps(data))
        #sleep
        sleep(5)

IndentationError: unexpected indent (2256739548.py, line 88)